In [1]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from kara.model import Sequential
from kara.layers import Dense, LeakyReLU, Dropout
from kara.wrapper import KerasRegressor

ModuleNotFoundError: No module named 'kara'

In [ ]:
# Load Dataset
def load_dataset(path):
    return pd.read_csv(path)

In [ ]:
# Prepprocess Dataset
def preprocess_dataset(num_cols, cat_cols):
    num_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    cat_pipe = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
         ('scaler', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    return ColumnTransformer(transformers=[
        ('num', num_pipe, num_cols),
        ('cat', cat_pipe, cat_cols)
    ])


In [ ]:
# Create Model
def create_model(input_shape, layers, activation, dropout_rate):
    
    model = Sequential()
    for i, layer_size in enumerate(layers):
        if i == 0:
            model.add(Dense(layer_size, input_shape=input_shape, activation=activation))
        else:
            model.add(Dense(layer_size, activation=activation))
        model.add(Dropout(dropout_rate))
        
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Hyperparameter Tuning
def hyperparameter_tuning(X_train, y_train, input_shape):
    model = KerasRegressor(build_fn=lambda: create_model(input_shape, layers=[128, 64], activation='relu', dropout_rate=0.2))
    param_grid = {
        'batch_size': [32, 64],
        'epochs': [100, 150]
    }
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
    grid_results = grid.fit(X_train, y_train)
    return grid_results.best_estimator_

In [ ]:
# Evaluate Model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return mse, r2


In [ ]:
if __name__ == '__main__':
    # Load Dataset
    df = load_dataset('../data/housing.csv')
    
    # Split features and target
    X = df.drop(columns=['Id', 'SalePrice'], axis=1)
    y = df['SalePrice']
    
    # Define numerical and categorical columns
    num_cols = X.select_dtypes(include=['numbers']).columns.to_list()
    cat_cols = X.select_dtypes(include=['object']).columns.to_list()
    
    # Prerocess features
    preprocessor = preprocess_dataset(num_cols, cat_cols)
    X_processed = preprocessor.fit_transform(X)
    
    # Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=0)
    
    # Hyperparameter Tuning
    model = hyperparameter_tuning(X_train, y_train, input_shape=(X.train.shape[1],))
    
    # Evaluate Model
    mse, r2 = evaluate_model(model, X_test, y_test)
    print(f' Test MSE: {mse:.2f}')
    print(f'Test R2: {r2:.2f}')
    